- 세그먼트 : 이미지를 여러개의 영역으로 나누는 작업으로 컴퓨터의 비전 및 이미지 처리에 사용
  - 이진 세그먼트
  - 다중 클래스 세그먼트
  - 인스턴스 세그먼트
    - 같은 클래스의 객체를 개별적으로 구분
  - 파노라믹 세그먼트
    - 인스턴스 세그먼트와 픽셀 분류의 조합, 모든 픽셀을 객체로 할당하거나 배경으로 분류

참고 자료 : https://www.tensorflow.org/tutorials/images/segmentation?hl=ko

In [1]:
import tensorflow as tf
data_url = "https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz"
data_dir = tf.keras.utils.get_file(origin=data_url, extract=True)

In [ ]:
import tarfile
import os
# Extract the .tar.gz file manually
if data_dir.endswith('.tar.gz'):
    # Open the downloaded .tar.gz file and extract it
    with tarfile.open(data_dir, "r:gz") as tar_ref:
        tar_ref.extractall(os.path.dirname(data_dir))

    # Check where the data was extracted
    extracted_dir = os.path.join(os.path.dirname(data_dir), 'images')
    print(f"Data extracted to: {extracted_dir}")
else:
    print("The downloaded file is not in the correct format.")

In [ ]:
os.listdir(extracted_dir)

In [ ]:
data_url = "http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz"
data_dir = tf.keras.utils.get_file(origin=data_url, extract=True)
import tarfile
import os
# Extract the .tar.gz file manually
if data_dir.endswith('.tar.gz'):
    # Open the downloaded .tar.gz file and extract it
    with tarfile.open(data_dir, "r:gz") as tar_ref:
        tar_ref.extractall(os.path.dirname(data_dir))

    # Check where the data was extracted
    extracted_dir_annotations = os.path.join(os.path.dirname(data_dir), 'annotations')
    print(f"Data extracted to: {extracted_dir_annotations}")
else:
    print("The downloaded file is not in the correct format.")

In [ ]:
os.listdir(extracted_dir_annotations+'/trimaps')

In [ ]:
input_dir = extracted_dir
target_dir = extracted_dir_annotations+'/trimaps'
img_size = (160,160)
num_classes = 4
batch_size = 32
input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".jpg")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

In [ ]:
print(len(input_img_paths), len(target_img_paths))

In [ ]:
[ (input_path, target_path) for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10])]

이미지확인

In [ ]:
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps

display(Image(filename=input_img_paths[0]))
img = PIL.ImageOps.autocontrast(load_img(target_img_paths[0]))
display(img)

In [ ]:
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
class OxfordPets(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""
    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size
    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i: i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i: i + self.batch_size]
        x = np.zeros((batch_size,) + self.img_size + (3,), dtype='float32')
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)
            x[j] = img
        y = np.zeros((batch_size,) + self.img_size + (1,), dtype='uint8')
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size,
                           color_mode='grayscale')
            y[j] = np.expand_dims(img, 2)
        return x, y

In [ ]:
from tensorflow.keras import layers

def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

        # Project residual
        residual = layers.Conv2D(
            filters, 1, strides=2, padding='same')(previous_block_activation)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    previous_block_activation = x  # Set aside residual

    for filters in [256, 128, 64, 32]:
        x = layers.Activation('relu')(x)
        x = layers.Conv2DTranspose(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation('relu')(x)
        x = layers.Conv2DTranspose(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding='same')(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(
        num_classes, 3, activation='sigmoid', padding='same')(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model


In [ ]:
# refresh RAM
keras.backend.clear_session()

In [ ]:
# model build
model = get_model(img_size, num_classes)
model.summary()

테이터 학습용과 검증용 분할

In [ ]:
import random
val_samples = 1000
random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_img_paths)
train_input_img_paths = input_img_paths[:-val_samples]
train_target_img_paths = target_img_paths[:-val_samples]
val_input_img_paths = input_img_paths[-val_samples:]
val_target_img_paths = target_img_paths[-val_samples:]

# 데이터 셋 만들기
train_gen = OxfordPets(  batch_size, img_size, train_input_img_paths, train_target_img_paths)
val_gen = OxfordPets(batch_size, img_size, val_input_img_paths, val_target_img_paths)

모델 학습

In [ ]:
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')
callbacks = [
    keras.callbacks.ModelCheckpoint('oxford_segmentation.keras', save_best_only=True)
]
epochs = 15
model.fit(train_gen, epochs=epochs, validation_data=val_gen, callbacks=callbacks)

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))